In [1]:
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
from keras.models import Sequential, load_model,Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from keras import initializers
from keras.initializers import RandomNormal
from matplotlib import pyplot
import numpy as np
from math import sqrt
import pickle

In [2]:
#chargement des données
with open('dataset_tau_array', 'rb') as f:
    X_train = pickle.load(f)
X_train = np.array(X_train)

In [3]:
with open('nom_colonne','rb') as f:
    nom_col = pickle.load(f)

In [4]:
#test,train
X_train,X_test = train_test_split(X_train,test_size = 0.2)

In [6]:
def build_autoencoder(in_shape=(2598, 21, 1), size_mini = 10) :
    autoencoder = Sequential()
    #encoder
    autoencoder.add(Input(shape=in_shape))
    autoencoder.add(Flatten())
    autoencoder.add(Dense(512))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dropout(0.3))
    autoencoder.add(Dense(256))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dropout(0.3))
    autoencoder.add(Dense(size_mini))
    #decoder
    autoencoder.add(BatchNormalization())
    autoencoder.add(Dense(256))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(400))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(512))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(2598 * 21 * 1,activation='sigmoid'))
    autoencoder.add(Reshape((2598, 21, 1)))
    autoencoder.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return autoencoder

In [7]:
autoencoder = build_autoencoder()

In [8]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 54558)             0         
                                                                 
 dense (Dense)               (None, 512)               27934208  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

In [9]:
#fitting même entrée que la sortie
autoencoder.fit(X_train,X_train,epochs=4, batch_size=128,validation_data = (X_test,X_test))

Epoch 1/4
10/10 [==============================] - 5s 414ms/step - loss: 2.8394 - accuracy: 0.8652 - val_loss: 0.7519 - val_accuracy: 0.9781
Epoch 2/4
10/10 [==============================] - 4s 390ms/step - loss: 0.8598 - accuracy: 0.9809 - val_loss: 0.5808 - val_accuracy: 0.9850
Epoch 3/4
10/10 [==============================] - 4s 392ms/step - loss: 0.3490 - accuracy: 0.9853 - val_loss: 0.5355 - val_accuracy: 0.9861
Epoch 4/4
10/10 [==============================] - 4s 391ms/step - loss: 0.3181 - accuracy: 0.9860 - val_loss: 0.4913 - val_accuracy: 0.9864


In [10]:
autoencoder.save('./exemple_VAE_model/model_VAE')

INFO:tensorflow:Assets written to: ./exemple_VAE_model/model_VAE\assets


In [29]:
autoencoder = load_model('./exemple_VAE_model/model_VAE')

In [11]:
#on récupère le decoder qui correspond aux dernières couches
decoder = Sequential()
decoder.add(Input(shape=10))
for layer in autoencoder.layers[8:]: decoder.add(layer)

In [12]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 10)               40        
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 256)               2816      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 400)               102800    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 400)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               205312    
                                                      

In [14]:
#On vient générer une entrée de nombre aléatoire pour que le generator tente de les transformer en image
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)  
    z_input = x_input.reshape(n_samples, latent_dim)
    return z_input

In [15]:
def to_sequence(prot_array,nom_col) :
    prot_aa = []
    for i in range(prot_array.shape[1]) :
        maxi = 0
        index = 0
        for j in range(prot_array[:,i,:,:].shape[1]) :
            if prot_array[0,i,j,0] > maxi :
                maxi = prot_array[:,i,j,:]
                index = j
        prot_aa.append(nom_col[index])
        prot_aa = [i for i in prot_aa if i != '-']
    return ''.join(prot_aa)

In [16]:
latent = generate_latent_points(10,1)
prot_array = decoder.predict(latent)
prot_seq = to_sequence(prot_array,nom_col)

In [17]:
prot_seq

'MAEPRQEFVMEDHAQGYTQDEGDDHGLKESPLQTPADDGSEEPGSETSDAKSTPTAEDTAPLVEGQAHEIPEGTTAEEAGIGDTPNLEDQAAGHVTQGARGKKKGADGKTGTKIATPRGAAPPGQKGQANATRIPAKTPPAPKTPPSSGESPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKKVAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDSNVQSKCGSKDNIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPGGGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSASLAKQGL'